# FigureYa25Plus_Sankey_py

**Author:** Ying Ge

In [5]:
from datetime import datetime
from IPython.display import Markdown

current_date = datetime.now().strftime("%Y-%m-%d")
Markdown(f"**Date**: {current_date}")

**Date**: 2025-10-18

## Academic Citation
If you use this code in your work or research, we kindly request that you cite our publication:

Xiaofan Lu, et al. (2025). FigureYa: A Standardized Visualization Framework for Enhancing Biomedical Data Interpretation and Research Efficiency. iMetaMed. https://doi.org/10.1002/imm3.70005

## 需求描述
## Requirement Description

使用plotly创建美观、交互式的Sankey图。

Draw interactive sankey with plotly.

![](example.png)

出自<https://www.nature.com/articles/s41388-019-1026-9>

Figure 2: Driver genes and pathways altered in MB.c, Graphical summary of the most frequently mutated genes (≥10 affected cases) and their subgroup distribution

## 应用场景
## Application Scenarios
一套数据集可能有多重属性，每层属性之间有交叉，就可以用这种图来展示。另外，以第一变量为参照，往后的多层变量均参考与第一变量的关系，从而展示第一变量的流向。

A dataset may have multiple attributes, and when there is an overlap between each layer of attributes, this type of plot can be used to display it. Additionally, using the first variable as a reference, subsequent multi-layer variables are all based on their relationship with the first variable, thereby illustrating the flow of the first variable.

## 环境设置
## Environment Configuration

In [6]:
#python --version
#pip install jupyter ipykernel pandas numpy matplotlib seaborn
#pip install scipy statsmodels plotly
#pip install notebook nbconvert

In [7]:
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo

## 输入文件
## Input files

easy_input.txt，至少要有两列，想画出几列就提供几列数据。此处第一列是基因，第二列是突变，第三列是癌症亚型；想画更多层次关系就继续向后添加列。

或者只写两列，第一列是miRNA，第二列是它的靶基因。

easy_input.txt, there should be at least two columns; provide as many columns of data as you want to draw. Here the first column is the gene, the second is the mutation, and the third is the cancer subtype; keep adding columns backwards if you want to draw more hierarchical relationships.

Or just write two columns, with the first column being the miRNA and the second column being its target gene.

## 开始画图
## Start plotting

In [8]:
# 设置 Plotly 在 Jupyter notebook 中显示 | Set up Plotly to display in Jupyter notebook
pyo.init_notebook_mode(connected=True)
 
# 读取数据 | Read data
df = pd.read_csv('easy_input.txt', sep='\t', index_col=0)
print("数据预览: | Data preview:")
print(df.head())
 
# 定义颜色方案 | Define color scheme
colors = ["#223D6C","#D20A13","#FFD121","#088247","#11AA4D","#58CDD9",
          "#7A142C","#5D90BA","#029149","#431A3D","#91612D","#6E568C",
          "#E0367A","#D8D155","#64495D","#7CC767"] * 3
 
def get_ordered_unique_values(series):
    """
    获取保持原始出现顺序的唯一值列表 | Get unique values list preserving original order
    """
    seen = set()
    result = []
    for item in series:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
 
def create_sankey_plotly_simple(df, columns=None, title="Sankey Diagram"):
    """
    创建响应式的 Plotly 桑基图，保持原始顺序 | Create responsive Plotly Sankey diagram preserving original order
    """
    if columns is None:
        columns = df.columns.tolist()
    
    # 为每列按原始出现顺序获取唯一值 | Get unique values for each column in original order
    column_categories = {}
    for col in columns:
        column_categories[col] = get_ordered_unique_values(df[col])
    
    # 创建全局节点列表 | Create global node list
    all_categories = []
    for col in columns:
        all_categories.extend(column_categories[col])
    
    # 创建节点映射 | Create node mapping
    node_dict = {cat: i for i, cat in enumerate(all_categories)}
    
    # 创建链接 | Create links
    links = []
    for i in range(len(columns) - 1):
        source_col = columns[i]
        target_col = columns[i + 1]
        
        # 计算每个组合的频次 | Calculate frequency for each combination
        combo_counts = df.groupby([source_col, target_col], sort=False).size().reset_index(name='count')
        
        for _, row in combo_counts.iterrows():
            source_idx = node_dict[row[source_col]]
            target_idx = node_dict[row[target_col]]
            value = row['count']
            
            links.append({
                'source': source_idx,
                'target': target_idx,
                'value': value
            })
    
    # 为节点分配颜色 | Assign colors to nodes
    node_colors = []
    color_idx = 0
    for col in columns:
        for cat in column_categories[col]:
            node_colors.append(colors[color_idx % len(colors)])
            color_idx += 1
    
    # 创建半透明的链接颜色 | Create semi-transparent link colors
    link_colors = []
    for link in links:
        # 使用源节点的颜色，但添加透明度 | Use source node color with transparency
        source_color = node_colors[link['source']]
        # 将十六进制颜色转换为 rgba | Convert hex color to rgba
        hex_color = source_color.lstrip('#')
        rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        rgba_color = f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, 0.4)'
        link_colors.append(rgba_color)
    
    # 创建 Plotly 桑基图 | Create Plotly Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_categories,
            color=node_colors
        ),
        link=dict(
            source=[link['source'] for link in links],
            target=[link['target'] for link in links],
            value=[link['value'] for link in links],
            color=link_colors
        )
    )])
    
    # 设置布局，优化在 notebook 中的显示 | Set layout optimized for notebook display
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,  # 标题居中 | Center title
            font=dict(size=16)
        ),
        font_size=12,
        # 为 notebook 显示设置合适的尺寸 | Set appropriate size for notebook display
        width=None,   # 让它自适应 notebook 宽度 | Let it adapt to notebook width
        height=500,   # 设置一个合理的高度 | Set reasonable height
        margin=dict(
            l=20,   # 左边距 | Left margin
            r=20,   # 右边距 | Right margin
            t=60,   # 上边距 | Top margin
            b=20    # 下边距 | Bottom margin
        )
    )
    
    return fig
 
def print_data_summary(df):
    """
    打印数据摘要信息 | Print data summary information
    """
    print("=== 数据摘要 | Data Summary ===")
    print(f"总记录数 | Total records: {len(df)}")
    print(f"基因数量 | Number of genes: {df['gene'].nunique()}")
    print(f"突变类型数量 | Number of mutation types: {df['mutation'].nunique()}")
    print(f"亚型数量 | Number of subtypes: {df['subtype'].nunique()}")
    
    print("\n各列的原始顺序 | Original order of each column:")
    for col in df.columns:
        unique_vals = get_ordered_unique_values(df[col])
        print(f"{col}: {unique_vals}")
 
# 执行分析和绘图 | Execute analysis and plotting
print_data_summary(df)

数据预览: | Data preview:
  gene mutation subtype
1   G4     mut1      S2
2   G4     mut3      S3
3   G3     mut5      S3
4   G2     mut5      S2
5   G8     mut4      S1
=== 数据摘要 | Data Summary ===
总记录数 | Total records: 100
基因数量 | Number of genes: 9
突变类型数量 | Number of mutation types: 6
亚型数量 | Number of subtypes: 4

各列的原始顺序 | Original order of each column:
gene: ['G4', 'G3', 'G2', 'G8', 'G6', 'G9', 'G5', 'G7', 'G1']
mutation: ['mut1', 'mut3', 'mut5', 'mut4', 'mut2', 'mut6']
subtype: ['S2', 'S3', 'S1', 'S4']


In [9]:
# 创建图形并在 notebook 中显示 | Create figures and display in notebook
print("创建完整的桑基图... | Creating complete Sankey diagram...")
fig1 = create_sankey_plotly_simple(df, title="Gene → Mutation → Subtype Flow")
# 在 notebook 中显示图形 | Display figure in notebook
fig1.show()

创建完整的桑基图... | Creating complete Sankey diagram...


In [10]:
print("创建基因到亚型的直接流向图... | Creating direct flow diagram from gene to subtype...")
fig2 = create_sankey_plotly_simple(df[['gene', 'subtype']], 
                                 title="Gene → Subtype Direct Flow")
# 在 notebook 中显示图形 | Display figure in notebook
fig2.show()

创建基因到亚型的直接流向图... | Creating direct flow diagram from gene to subtype...


In [11]:
# 可选：如果你还想保存 HTML 文件 | Optional: if you want to save HTML files
print("保存为独立的 HTML 文件... | Saving as standalone HTML files...")
 
config = {
    'responsive': True,
    'displayModeBar': True,
    'displaylogo': False
}
 
fig1.write_html("sankey_full.html", config=config)
fig2.write_html("sankey_direct.html", config=config)
 
print("=== 保存完成 | Save Complete ===")
print("- 图形已在 notebook 中显示 | Figures displayed in notebook")
print("- 同时保存了独立的 HTML 文件 | Standalone HTML files also saved")

保存为独立的 HTML 文件... | Saving as standalone HTML files...
=== 保存完成 | Save Complete ===
- 图形已在 notebook 中显示 | Figures displayed in notebook
- 同时保存了独立的 HTML 文件 | Standalone HTML files also saved


## Session Info

In [12]:
import IPython
print(IPython.sys_info())

!jupyter nbconvert --to html FigureYa25Plus_Sankey_py.ipynb

{'commit_hash': '48b00eb07',
 'commit_source': 'installation',
 'default_encoding': 'utf-8',
 'ipython_path': '/Users/air/Library/Python/3.14/lib/python/site-packages/IPython',
 'ipython_version': '9.6.0',
 'os_name': 'posix',
 'platform': 'macOS-14.8.1-x86_64-i386-64bit-Mach-O',
 'sys_executable': '/usr/local/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '3.14.0 (v3.14.0:ebf955df7a8, Oct  7 2025, 08:20:14) [Clang '
                '16.0.0 (clang-1600.0.26.6)]'}
[NbConvertApp] Converting notebook FigureYa25Plus_Sankey_py.ipynb to html
[NbConvertApp] Writing 330189 bytes to FigureYa25Plus_Sankey_py.html
